This notebook is used for drawing intial ROIs (V1-hV4) that we use in retinotopic cleaning and to run the retinotopic cleaning then inspect the results. After this we can run the bayesian retinotopic maps using neuropythy.

1. Visualise maps
2. Draw ROIs
3. Run cleaning
4. Inspect results

First, we will import and configure a range of libararies that neuropythy depends upon

In [3]:
# Import some standard/utility libraries:
import os, sys, time, h5py, zipfile
import six           # six provides python 2/3 compatibility

# Import our numerical/scientific libraries, scipy and numpy:
import numpy as np
import scipy as sp

# The pimms (Python Immutables) library is a utility library that enables lazy
# computation and immutble data structures; https://github.com/noahbenson/pimms
import pimms

# The neuropythy library is a swiss-army-knife for handling MRI data, especially
# anatomical/structural data such as that produced by FreeSurfer or the HCP.
import neuropythy as ny

# Import graphics libraries:
# Matplotlib/Pyplot is our 2D graphing library:
import matplotlib as mpl
import matplotlib.pyplot as plt
# We also use the 3D graphics library ipyvolume for 3D surface rendering
import ipyvolume as ipv

/Users/et2160/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'ipyvolume'

In [ ]:
# These "magic commands" tell matplotlib that we want to plot figures inline and
# That we are using qt as a backend; due to bugs in certain versions of
# matplotlib, we put them in a separate cell from the import statements above
# and the configuration statements below.
%gui qt
%matplotlib inline

In [ ]:
# Additional matplotlib preferences:
font_data = {'family':'sans-serif',
             'sans-serif':['Helvetica Neue', 'Helvetica', 'Arial'],
             'size': 10,
             'weight': 'light'}
mpl.rc('font',**font_data)
# we want relatively high-res images, especially when saving to disk.
mpl.rcParams['figure.dpi'] = 72*2
mpl.rcParams['savefig.dpi'] = 72*4

Set up paths to data in Retinotopy_Comp in the winawerlab server

In [ ]:
project_path = '/Volumes/server/Projects/attentionpRF' #path to BIDS directory

subname = 'wlsubj122' #what subject?
sesname = 'nyu3t01' #and session

# The subject freesurfer directory and the kind of subject it is
subject_path = project_path + '/derivatives/freesurfer/sub-' + subname
subject_type = 'freesurfer' # where the freesurfer data are stored

# The directory containing the pRF parameters
#prf_path = project_path + '/derivatives/prfs/sub-' + subname + '/ses-' + sesname
prf_path = project_path + '/derivatives/prfs/sub-' + subname + '/ses-nyu3t99/prfFolder/avg'
# how the files are named:
prf_filename_patt = '{hemi}.{prop}.mgz' #pRF

# the prf property names as keys, what are the {prop} taks in the file?
# we are using the adjusted angle params that we converted from vista to neuropythy
prf_props = {'polar_angle':        'angle_adj',
             'eccentricity':       'eccen',
             'radius':             'sigma',
             'variance_explained': 'vexpl'}

In [ ]:
project_path = '/Volumes/server/Projects/Retinotopy_NYU_3T' #path to BIDS directory

subname = 'wlsubj123' #what subject?
sesname = 'nyu3t01' #and session

# The subject freesurfer directory and the kind of subject it is
subject_path = project_path + '/derivatives/freesurfer/sub-' + subname
subject_type = 'freesurfer' # where the freesurfer data are stored

# The directory containing the pRF parameters
#prf_path = project_path + '/derivatives/prfs/sub-' + subname + '/ses-' + sesname
prf_path = project_path + '/derivatives/prfanalyze-vista/sub-' + subname + '/ses-nyu3t01/'
# how the files are named:
prf_filename_patt = '{hemi}.{prop}.mgz' #pRF

# the prf property names as keys, what are the {prop} taks in the file?
# we are using the adjusted angle params that we converted from vista to neuropythy
prf_props = {'polar_angle':        'angle_adj',
             'eccentricity':       'eccen',
             'radius':             'sigma',
             'variance_explained': 'vexpl'}

In [ ]:
# Load the prf parameters into a map, using the data in the above cell.
prf_params = {}
for hemi in ['lh','rh']:
    tmp = {}
    for (prop, fname_tag) in prf_props.items():
        filename = prf_filename_patt.format(hemi=hemi, prop=fname_tag)
        tmp['prf_' + prop] = ny.load(os.path.join(prf_path, filename))
    prf_params[hemi] = tmp
    
    # Print them out to be safe
prf_params

In [ ]:
# we >copy< each hemisphere surface so we can project the retinotopy data onto mesh for us to draw ROIs on

#first, set up a subject with the above data included
sub = ny.freesurfer_subject(subject_path)
lh = sub.lh.with_prop(prf_params['lh'])
rh = sub.rh.with_prop(prf_params['rh'])
sub = sub.copy(hemis=pimms.assoc(sub.hemis, lh=lh, rh=rh))

In [ ]:
subject_path

In [ ]:
flatmaps = {h: ny.to_flatmap('occipital_pole', sub.hemis[h])
            for h in ('lh', 'rh')}

## Tracing ROIs

Now it is time to draw the ROIs by tracing a >trace<, saving it out as a JSON, and then we convert the trace to a >path< to be projected onto the cortical surface...

The way the following cell works is that we evaluate it once for each of the ROIs we're going to draw. We draw V1 first then V2, then V3, then hV4. We update the name of the ROI we're drawing (at the top of the cell) each time we draw a new ROI (so that the resulting trace gets stored in a new place).

Note that eventually we turn these paths into labels, and the higher order labels exclude the lower order labels, so we can overlap our ROIs...


Note: sometimes I get errors when loading up the flatmap to draw the ROIs on. The errors tell me nothing but if you just try to reload the flatmap it works. Some bug.

In [ ]:
%matplotlib notebook

In [ ]:
h = 'lh' 
roi = 1
number_of_rois = 7

mask=({'and':[('prf_variance_explained', 0, np.inf),('prf_eccentricity', 0, np.inf)]})

if roi == 1: traces = {}
flatmap = flatmaps[h]
hemi = sub.hemis[h]

# Go ahead and setup the figure and axes onto which we will plot:
(fig,ax) = plt.subplots(1,1, figsize=(5,5), dpi=2*72)
fig.subplots_adjust(0,0,1,1,0,0)
ax.axis('off')

# The trace_roi function will take care of the plots (which we made
# in an earlier cell) and also add the previous ROI traces:
for (k,v) in six.iteritems(traces):
    if v is None or not v.is_persistent(): continue
    (x,y) = v.points
    # Close the points in the trace:
    x = np.concatenate([x,[x[0]]])
    y = np.concatenate([y,[y[0]]])
    # Plot the trace:
    ax.plot(x, y, 'k.-', lw=0.5)

# Make the plots we want to be able to cycle through
plots = [ny.cortex_plot(flatmap, axes=ax, color='prf_polar_angle', mask = mask),
         ny.cortex_plot(flatmap, axes=ax, color='prf_eccentricity', mask = mask),
         ny.cortex_plot(flatmap, axes=ax, color='prf_radius', mask = mask),
         ny.cortex_plot(flatmap, axes=ax)]


roi_path = ('/Volumes/server/Projects/attentionpRF/derivatives/freesurfer/sub-{}/surf/{}.ROIs_V1-IPS.mgz'.format(subname, h))
#roi_path = ('/Volumes/server/Projects/Retinotopy_NYU_3T/derivatives/prfanalyze-vista/sub-wlsubj135/ses-nyu3t01/{}.ROIs_V1-4.mgz'.format(h))
(u,v) = flatmaps[h].tess.indexed_edges
atlas = sub.load(roi_path)
lbls = atlas[flatmaps[h].labels]
ii = lbls[u] != lbls[v]
(x,y) = np.mean(flatmaps[h].edge_coordinates[:,:,ii], axis=0)
ax.plot(x, y,'.',markersize=0.5,color='white')



tr = ny.graphics.trace_roi(hemi, flatmap, ax, closed=True, plot_list=plots)
traces[roi] = tr
plt.show()





In [ ]:
    # Once the traces have been made, we can export them to a JSON file;
# this saves all the relevant data (including the parameters of the
# map-projection used to make the plots). They can be loaded back
# in later using ny.load(filename).

# JSON doesn't support dictionary keys that are integers, so we
# convert them into strings before we write out the file:
#json_traces = {('V%d' % k): traces for (k,tr) in six.iteritems(traces)}
json_traces = {('V%d' % k): tr for (k,tr) in six.iteritems(traces)}


ny.save('/Volumes/server/Projects/attentionpRF/derivatives/ROI_traces/v1234567-traces_sub-{}_{}.json'.format(subname,h), json_traces)

In [ ]:
rh_traces = ny.load('/Volumes/server/Projects/attentionpRF/derivatives/ROI_traces/v1234567-traces_sub-{}_rh.json'.format(subname))
number_of_rois = 6
rh_old_trace = {}
for t in range(1,number_of_rois+1):
    rh_old_trace[t] = rh_traces['V{}'.format(t)]
rh_traces = rh_old_trace

In [ ]:
rh_paths = {k: v.to_path(sub.hemis['rh']) for (k,v) in six.iteritems(rh_traces)}

rh_lbls = ny.paths_to_labels(sub.hemis['rh'], rh_paths)

save_dir =  '/Volumes/server/Projects/attentionpRF/derivatives/freesurfer/sub-' + subname + '/surf'
ny.save(save_dir + '/rh.ROIs_V1-IPS.mgz'.format(subname), rh_lbls)




In [ ]:
# Now let's turn the traces into paths on the cortical surface, and then V1-V3 labels.

#Load in our ROI traces
lh_traces = ny.load('/Volumes/server/Projects/attentionpRF/derivatives/ROI_traces/v1234567-traces_sub-{}_lh.json'.format(subname))
rh_traces = ny.load('/Volumes/server/Projects/attentionpRF/derivatives/ROI_traces/v1234567-traces_sub-{}_rh.json'.format(subname))

# now we want to convert the trace dictionary key from V1 - V3 to just 1 - 3 so that the cleaning can read it properly
lh_old_trace = {}
for t in range(1,number_of_rois+1):
    lh_old_trace[t] = lh_traces['V{}'.format(t)]
lh_traces = lh_old_trace

rh_old_trace = {}
for t in range(1,number_of_rois+1):
    rh_old_trace[t] = rh_traces['V{}'.format(t)]
rh_traces = rh_old_trace


In [ ]:
#and finally convert the traces to paths for the cortical surface

# Turn these traces into paths for the cortical surface. 
lh_paths = {k: v.to_path(sub.hemis['lh']) for (k,v) in six.iteritems(lh_traces)}
rh_paths = {k: v.to_path(sub.hemis['rh']) for (k,v) in six.iteritems(rh_traces)}

# The paths get turned into labels; higher-number labels exclude the
# vertices in lower-number labels, so the overlapping areas go to V1
# first, V2 second.
lh_lbls = ny.paths_to_labels(sub.hemis['lh'], lh_paths)
rh_lbls = ny.paths_to_labels(sub.hemis['rh'], rh_paths)

In [ ]:
save_dir =  '/Volumes/server/Projects/attentionpRF/derivatives/freesurfer/sub-' + subname + '/surf'
ny.save(save_dir + '/lh.ROIs_V1-IPS.mgz'.format(subname), lh_lbls)
ny.save(save_dir + '/rh.ROIs_V1-IPS.mgz'.format(subname), rh_lbls)

